<h1> Classification with a Neural Network using Keras (Sequential API)</h1>

<h2>1. Imports and load data</h2>

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.regularizers import l2
from aeon.classification.deep_learning import TimeCNNClassifier
from tensorflow.keras.layers import Input, Dense, Conv1D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

In [6]:
class DataProcessor:
    def __init__(self, input_path, file_names):
        self.input_path = input_path
        self.file_names = file_names
        
    def read_files(self):
        self.data = {}
        print("Reading files...")
        for file in self.file_names:
            with open(self.input_path + file + '.txt', 'r') as f:
                self.data[file] = pd.read_csv(f, header=None, sep='\t')
        return self.data
    
    def print_shape(self):
        print("Files read:")
        for file in self.data:
            print(f"{file}: {self.data[file].shape}")
            
    def create_target_df(self):
        target_columns = ['Cooler_Condition', 'Valve_Condition', 
                        'Internal_Pump_Leakage', 'Hydraulic_Accumulator', 
                        'Stable_Flag']
        self.data['target'].columns = target_columns
        self.valve_condition = self.data['target']['Valve_Condition']
        #del self.data['target']
        return self.valve_condition

def process_data():
    input_path = "input_data/"
    file_names = [
        "ce", "cp", "eps1", "se", "vs1", 
        "fs1", "fs2", 
        "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
        "ts1", "ts2", "ts3", "ts4", "target"
    ]
    
    processor = DataProcessor(input_path, file_names)
    data = processor.read_files()
    processor.print_shape()
    df_target = processor.create_target_df()
    df_target = processor.valve_condition
    return data, df_target

data, df_target = process_data()

Reading files...
Files read:
ce: (2205, 60)
cp: (2205, 60)
eps1: (2205, 6000)
se: (2205, 60)
vs1: (2205, 60)
fs1: (2205, 600)
fs2: (2205, 600)
ps1: (2205, 6000)
ps2: (2205, 6000)
ps3: (2205, 6000)
ps4: (2205, 6000)
ps5: (2205, 6000)
ps6: (2205, 6000)
ts1: (2205, 60)
ts2: (2205, 60)
ts3: (2205, 60)
ts4: (2205, 60)
target: (2205, 5)


<h2>2. Create input and target data </h2>

We use the six sensors which we identified as relevant during data exploration: 'eps1', 'se', 'fs1', 'ps1', 'ps2', 'ps3' 

In [7]:
df_list = ['eps1', 'se', 'fs1', 'ps1', 'ps2', 'ps3']
input_df = pd.concat([data[i] for i in df_list], axis = 1)
input_df

,0,1,2,3,4,5,6,7,8,9,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
0,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2411.6,2409.6,...,2.336,2.391,2.375,2.297,2.328,2.383,2.328,2.250,2.250,2.211
1,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,2409.6,...,2.297,2.266,2.266,2.219,2.211,2.266,2.273,2.211,2.195,2.219
2,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2397.8,2395.8,...,2.359,2.391,2.391,2.375,2.375,2.375,2.305,2.305,2.320,2.266
3,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2383.8,2382.8,2382.8,...,2.117,2.219,2.281,2.227,2.164,2.164,2.219,2.250,2.273,2.273
4,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2372.0,2373.0,...,2.141,2.172,2.187,2.227,2.219,2.211,2.242,2.219,2.227,2.297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,2416.4,...,2.328,2.305,2.328,2.359,2.375,2.281,2.242,2.250,2.266,2.273
2201,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,2415.6,...,2.273,2.383,2.359,2.297,2.297,2.336,2.406,2.461,2.461,2.406
2202,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,...,2.227,2.242,2.219,2.211,2.273,2.273,2.250,2.219,2.219,2.250
2203,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,2413.6,...,2.328,2.328,2.328,2.281,2.266,2.305,2.281,2.250,2.242,2.281


Standardise the input and target data

In [8]:
# Standardise the target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df_target)

# Standatdise the input
scaler = StandardScaler()
input_data_scaled = scaler.fit_transform(input_df)

In [9]:
input_data_scaled.shape

(2205, 24660)

<h2>3. Create the model, train it & make predictions </h2>

<ul>
<li>We create a neural network with 1 input layer, 2 hidden layers and 1 output layer --> 4 layers total
<li>We use the stochastic gradient descent with a middle-sized batch size of 32 since we dont have a very big data set
<li>We use SparseCrossEntropyLoss() for calculcating the loss. We use it because we have more than one two label classes
<li>In the output Layer we use the softmax activation function in order to get the probabilites for classification
</ul>

In [10]:
states = [27, 6728, 49122]
accs = []

In [11]:
X_train, X_temp, y_train, y_temp = train_test_split(input_data_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [16]:
cnn = TimeCNNClassifier(
    n_layers = 3,
    n_filters=3,
    kernel_regularizer=l2(0.01),
    n_epochs=100, 
    batch_size=32,
    loss = "binary_crossentropy",
    verbose = True
)  
cnn.fit(X_train, y_train)


NameError: name 'keras' is not defined

In [27]:
y_pred = cnn.predict(X_test)
# print the classification report for the test set
print(classification_report(y_test, y_pred))

# calculate accuracy and append to the list for later mean and std calculation
accuracy = accuracy_score(y_test, y_pred)
accs.append(accuracy) 

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
              precision    recall  f1-score   support

           0       0.88      1.00      0.94        36
           1       0.86      0.83      0.85        36
           2       0.96      0.75      0.84        36
           3       0.97      1.00      0.98       113

    accuracy                           0.93       221
   macro avg       0.92      0.90      0.90       221
weighted avg       0.93      0.93      0.93       221



In [26]:
for RANDOM_STATE in states:
    # split into train, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(input_data_scaled, y_encoded, test_size=0.2, random_state=RANDOM_STATE, stratify=y_encoded)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=RANDOM_STATE, stratify=y_temp)

    # initiating the model with Keras sequential API
    model = Sequential()

    # creating the input layer
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))

    # creating the hidden layers
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    # creating the output layer
    model.add(Dense(4, activation='softmax'))  # 4 labels and therefore 4 neurons in the output layer with softmax activation function

    # compile the model using the adam optimizer and the sparse categorical crossentropy loss function
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss=SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    # creating the early stopping callback using the validation loss as the monitoring parameter
    early_stopping = EarlyStopping(monitor='val_loss',    
                                   patience=10,           
                                   restore_best_weights=True,  
                                   verbose=1)

    # train the model
    model.fit(X_train, y_train, 
              epochs=100, 
              batch_size=32, 
              validation_data=(X_val, y_val),  #  validation set for monitoring during training
              verbose=0, 
              callbacks=[early_stopping])

    # nake predictions and evaluate the model
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    # print the classification report for the test set
    print(f"Classification Report for random state {RANDOM_STATE}:")
    print(classification_report(y_test, y_pred))
    
    # calculate accuracy and append to the list for later mean and std calculation
    accuracy = accuracy_score(y_test, y_pred)
    accs.append(accuracy)

# calculate mean and std of accuracy scores
accs_mean = round(np.mean(accs), 4)
accs_std = round(np.std(accs), 4)

print(f"Mean Accuracy: {accs_mean}")
print(f"Std Accuracy: {accs_std}")


c:\Users\Daniel\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 52: early stopping
Restoring model weights from the end of the best epoch: 42.
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Classification Report for random state 27:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00        36
           2       1.00      1.00      1.00        36
           3       1.00      1.00      1.00       113

    accuracy                           1.00       221
   macro avg       1.00      1.00      1.00       221
weighted avg       1.00      1.00      1.00       221



c:\Users\Daniel\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 71: early stopping
Restoring model weights from the end of the best epoch: 61.
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Classification Report for random state 6728:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       0.97      1.00      0.99        36
           2       1.00      0.94      0.97        36
           3       0.99      1.00      1.00       113

    accuracy                           0.99       221
   macro avg       0.99      0.99      0.99       221
weighted avg       0.99      0.99      0.99       221



c:\Users\Daniel\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 55: early stopping
Restoring model weights from the end of the best epoch: 45.
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Classification Report for random state 49122:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      0.97      0.99        36
           2       0.97      1.00      0.99        36
           3       1.00      1.00      1.00       113

    accuracy                           1.00       221
   macro avg       0.99      0.99      0.99       221
weighted avg       1.00      1.00      1.00       221

Mean Accuracy: 0.9955
Std Accuracy: 0.0037
